In [ ]:
import numpy as np
from numba import jit
from AFM_sinc import MDR_SLS_sinc, SLS_parabolic_LR_sinc, Hertzian_sinc
import matplotlib.pyplot as plt
from AFM_calculations import derivative_cd, av_dt
%matplotlib inline


A = -1.36e-9  #amplitude of the sinc excitation
R = 10.0e-9  #radius of curvature of the parabolic tip apex
startprint = 0.0
simultime = 1200.0e-6 #total simulation time
fo1 =20.0e3  #cantilever 1st mode resonance frequency
omega = 2.0*np.pi*fo1
period1 = 1.0/fo1  #fundamental period
to =7.0*period1   #centered time of the sinc excitation
fo2 = 6.27*fo1
fo3 = 17.6*fo1
Q1 = 2.0 #cantilever's 1st mode quality factor
Q2 = 8.0
Q3 = 12.0
BW = 2.5*fo1*2.0  #excitation bandwith of sinc function
k_m1 =  0.25 #cantilever's 1st mode stiffness
zb = 3.85e-9  #cantilever equilibrium position
period2 = 1.0/fo2
period3 = 1.0/fo3
dt= period3/1.0e5 #simulation timestep
printstep = dt*10.0 #timestep in the saved time array

In [ ]:
nu = 0.5  #time independent Poisson's ratio
G_v = 1.0e-1/(1.2*R)    #modulus of the spring in the Voigt unit that is in series with the upper spring

Gg_v = 100.0e9 /(2*(1+nu))  #Glassy modulus in the Voigt-SLS configuration reported in the grid
Jg = 1.0/Gg_v  #glassy compliance
tau_v = 0.01/omega  #retardation time reported in simulation grid
J = 1.0/G_v   #compliance of the spring in the Voigt unit that is in series with the upper spring
eta = tau_v*G_v
phi = 1.0/eta #fluidity of the dashpot in the Voigt unit that is in series with the upper spring
Je = J+Jg

# Now converting to the Maxwell SLS configuration: spring in parallel with Maxwell unit, note that these two models are mechanical analogs showing quantitatively the same behavior
Ge = 1.0/(Je)
G = J/(Jg*Je) 
Gg = (G+Ge)
tau_m = tau_v*(Ge/Gg)
eta = tau_m*G


In [ ]:
hertz_jit = jit()(Hertzian_sinc)
zb = 3.0e-9
%time t_h, tip_h, Fts_h = hertz_jit(A, to, BW, Gg, R, dt, startprint, simultime, fo1, k_m1, zb, printstep, Q1, Q2, Q3, nu)

In [ ]:
plt.plot( (t_h-to)*1.0e3, tip_h*1.0e9)
plt.xlim(0,0.01)
plt.ylim(-0.5,0.0)

In [ ]:
zb = 3.0e-9
MDR_jit = jit()(MDR_SLS_sinc)
%time t_m, tip_m, Fts_m, ca_m, sample_m = MDR_jit(A, to, BW, G, tau_m, R, dt, startprint, simultime, fo1, k_m1, zb, printstep, Ge, Q1, Q2, Q3, nu, 1000, 1.0e-9)
 

In [ ]:
plt.plot( (t_m-to)*1.0e3, tip_m*1.0e9)
plt.plot( (t_m-to)*1.0e3, sample_m*1.0e9)
plt.xlim(0,0.03)
plt.ylim(-0.5,0.5)

In [ ]:
zb = 4.0e-9
MDR_jit = jit()(MDR_SLS_sinc)
t_m2, tip_m2, Fts_m2, ca_m2, sample_m2 = MDR_jit(A, to, BW, G, tau_m, R, dt, startprint, simultime, fo1, k_m1, zb, printstep, Ge, Q1, Q2, Q3, nu, 1000, 1.0e-9)
  

In [ ]:
plt.plot( (t_m2-to)*1.0e3, tip_m2*1.0e9)
plt.plot( (t_m2-to)*1.0e3, sample_m2*1.0e9)
plt.xlim(0,0.03)
plt.ylim(-0.5,0.5)